In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
proj_dir = os.path.split(os.getcwd())[0]

%matplotlib inline

audit_df = pd.read_pickle(f'{proj_dir}/data/cleaned/audit.pkl')
auth_df = pd.read_pickle(f'{proj_dir}/data/cleaned/auth.pkl')
net_df = pd.read_pickle(f'{proj_dir}/data/cleaned/netflow.pkl')
win_df = pd.read_pickle(f'{proj_dir}/data/cleaned/win-security.pkl')